# faiss

> faiss backend API

In [ ]:
#| default_exp backends.faiss

In [ ]:
#| hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from emb_opt.imports import *
from emb_opt.core import QueryResult, VectorDatabase, dataset_from_query_results
try:
    import faiss
except:
    warnings.warn('faiss import failed - check if package is installed')

/home/dmai/miniconda3/envs/emb_opt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Faiss Backend

`FaissDatabase` integrates with a [faiss](https://github.com/facebookresearch/faiss) index.

`search_params` can be any params object compatible with faiss `search`

In [ ]:
#| export

class FaissDatabase(VectorDatabase):
    'Faiss backend'
    def __init__(self, 
                 faiss_index: faiss.Index, # faiss index to query
                 k: int, # query will return `k` values per query vector
                 search_params: Optional[faiss.SearchParameters]=None # search params for `faiss_index.search`
                ):
        
        self.faiss_index = faiss_index
        self.k = k
        self.search_params = search_params
        
    def query(self, query_vectors: np.ndarray) -> Dataset:
        distances, indices = self.faiss_index.search(query_vectors, self.k, params=self.search_params)
        
        n_queries, n_results = indices.shape
        
        results = []
        for query_idx in range(n_queries):
            embeddings = self.faiss_index.reconstruct_batch(indices[query_idx])
            
            for result_idx in range(n_results):
                db_idx = indices[query_idx, result_idx]
                embedding = embeddings[result_idx]
                distance = distances[query_idx, result_idx]
                
                if db_idx != -1:
                    result = QueryResult(query_idx, db_idx, embedding, distance, {})
                    results.append(result)
                
        return dataset_from_query_results(results)

In [ ]:
vectors = np.random.randn(64, 256)
index = faiss.IndexFlatL2(256)
index.add(vectors)
faiss_db = FaissDatabase(index, 5)

query_vecs = np.random.randn(2, 256)

query_dataset = faiss_db.query(query_vecs)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()